In [39]:
import pandas as pd

In [40]:
data_scrap = pd.read_csv('../../data/scrapped/avito_cars_scrapping.csv')

In [41]:
#shape of our scrapped data
data_scrap.shape

(11252, 14)

In [42]:
data_scrap.head()

,Date Pub,Ville,Année-Modèle,Boite de vitesses,Type de carburant,Kilométrage,Marque,Modèle,Nombre de portes,Origine,Première main,Puissance fiscale,État,Prix
0,il y a 2 heures,Casablanca,2018,Automatique,Essence,100 000 - 109 999,Fiat,500,5.0,WW au Maroc,Oui,7 CV,Excellent,107 000 DH
1,il y a 6 heures,Rabat,2022,Automatique,Diesel,120 000 - 129 999,BMW,X5,5.0,WW au Maroc,Oui,12 CV,Excellent,590 000 DH
2,il y a 6 heures,Marrakech,2014,Automatique,Diesel,120 000 - 129 999,Jeep,Grand Cherokee,5.0,Dédouanée,Non,12 CV,Excellent,165 000 DH
3,il y a 6 heures,Fès,2021,Manuelle,Diesel,90 000 - 94 999,Renault,Clio,5.0,WW au Maroc,Oui,6 CV,Excellent,128 000 DH
4,il y a 6 heures,Agadir,2020,Automatique,Diesel,140 000 - 149 999,Skoda,Karoq,5.0,Dédouanée,Oui,6 CV,Excellent,278 000 DH


In [43]:
#remove the column Date_pub
data_scrap = data_scrap.drop(columns=['Date Pub','État'])

In [76]:
data_scrap.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4257 entries, 0 to 11251
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Ville              2602 non-null   object 
 1   Année-Modèle       4257 non-null   int32  
 2   Boite de vitesses  4257 non-null   object 
 3   Type de carburant  4257 non-null   object 
 4   Kilométrage        4257 non-null   float64
 5   Marque             4257 non-null   object 
 6   Modèle             4257 non-null   object 
 7   Nombre de portes   4257 non-null   float64
 8   Origine            4257 non-null   object 
 9   Première main      4257 non-null   object 
 10  Puissance fiscale  4257 non-null   int32  
 11  Prix               4257 non-null   int64  
dtypes: float64(2), int32(2), int64(1), object(7)
memory usage: 399.1+ KB


In [45]:
#number of duplicates
data_scrap.duplicated().sum()

5604

In [46]:
#drop duplicates
data_scrap.drop_duplicates(inplace=True)

In [47]:
#convert the column Année-Modèle to a num
data_scrap['Année-Modèle'] = pd.to_numeric(data_scrap['Année-Modèle'], errors='coerce').fillna(0).astype(int)

In [48]:
#remove the CV in the column Puissange fiscale  
data_scrap['Puissance fiscale']=data_scrap['Puissance fiscale'].astype(str).str.split().apply(lambda x : x[0] 
                                                                                              if len(x[0])>0 else None)
#convert to the column to num
data_scrap['Puissance fiscale'] = pd.to_numeric(data_scrap['Puissance fiscale'], errors='coerce')

In [49]:
#remove DH from price
data_scrap['Prix']=data_scrap['Prix'].astype(str).str.replace('DH','').str.replace(r'\s+','', regex=True)
#convert to the column to num
data_scrap['Prix'] = pd.to_numeric(data_scrap['Prix'], errors='coerce')

In [50]:
#convert the column Kilometrage to a single value
data_scrap['Kilométrage'] = data_scrap['Kilométrage'].astype(str).str.replace(' ','').str.split('-')
.apply(lambda x :(int(x[0])+int(x[1]))/2 if len(x)==2 else None)

In [52]:
#name of columns
data_scrap.columns.to_list()

['Ville',
 'Année-Modèle',
 'Boite de vitesses',
 'Type de carburant',
 'Kilométrage',
 'Marque',
 'Modèle',
 'Nombre de portes',
 'Origine',
 'Première main',
 'Puissance fiscale',
 'Prix']

In [51]:
#number of nan values in each column
data_scrap.isna().sum()

Ville                2294
Année-Modèle            0
Boite de vitesses      41
Type de carburant      41
Kilométrage            70
Marque                 41
Modèle                 41
Nombre de portes      899
Origine               945
Première main        1028
Puissance fiscale     269
Prix                    0
dtype: int64

In [53]:
#remove the none values 
data_scrap = data_scrap.dropna(subset=['Année-Modèle','Boite de vitesses','Type de carburant','Kilométrage',
'Marque','Modèle','Origine','Première main','Puissance fiscale'])

(4332, 12)

In [54]:
#unique values of 
data_scrap['Type de carburant'].unique()

array(['Essence', 'Diesel', 'Hybride', 'Electrique', 'LPG'], dtype=object)

In [55]:
# Liste des modèles ayant 3 portes
models_with_3_doors = data_scrap[data_scrap['Nombre de portes']==3]['Modèle'].unique()

# Remplir les valeurs manquantes pour ces modèles
data_scrap.loc[
    (data_scrap['Modèle'].isin(models_with_3_doors)) & 
    (data_scrap['Nombre de portes'].isna()),
    'Nombre de portes'
] = 3


In [56]:
# Récupérer la liste des modèles avec 5 portes
models_with_5_doors = data_scrap[data_scrap['Nombre de portes']==5]['Modèle'].unique()

# Remplir les valeurs manquantes
data_scrap.loc[
    (data_scrap['Modèle'].isin(models_with_5_doors)) & 
    (data_scrap['Nombre de portes'].isna()),
    'Nombre de portes'
] = 5

In [57]:
#remove the rows where theres left none values in nombre de portes
data_scrap=data_scrap.dropna(subset=['Nombre de portes'])

In [58]:
data_scrap.shape

(4327, 12)

In [59]:
data_scrap[data_scrap['Modèle']=='R4']

,Ville,Année-Modèle,Boite de vitesses,Type de carburant,Kilométrage,Marque,Modèle,Nombre de portes,Origine,Première main,Puissance fiscale,Prix
4960,Marrakech,1994,Manuelle,Essence,324999.5,Renault,R4,5.0,Dédouanée,Non,6.0,13000
10865,NaN,1992,Manuelle,Essence,274999.5,Renault,R4,5.0,Dédouanée,Non,5.0,30000


In [60]:
#see number of the rows where price <20000
data_scrap[data_scrap['Prix']<20000]

,Ville,Année-Modèle,Boite de vitesses,Type de carburant,Kilométrage,Marque,Modèle,Nombre de portes,Origine,Première main,Puissance fiscale,Prix
507,Casablanca,2006,Manuelle,Diesel,67499.5,Toyota,Hilux,3.0,Dédouanée,Non,12.0,11
4699,Casablanca,2007,Manuelle,Diesel,224999.5,Renault,Megane,5.0,WW au Maroc,Non,6.0,6900
4701,Casablanca,2018,Automatique,Essence,104999.5,Ford,Ecosport,5.0,WW au Maroc,Non,6.0,132
4960,Marrakech,1994,Manuelle,Essence,324999.5,Renault,R4,5.0,Dédouanée,Non,6.0,13000
4965,Rabat,2019,Manuelle,Essence,2499.5,Peugeot,208,5.0,WW au Maroc,Non,6.0,12000
5335,Casablanca,2018,Automatique,Diesel,154999.5,Volkswagen,Golf 7,5.0,WW au Maroc,Non,8.0,1
6530,Casablanca,2006,Manuelle,Essence,27499.5,Suzuki,Maruti,5.0,Pas encore dédouanée,Non,5.0,18000
6568,Larache,2007,Manuelle,Essence,12499.5,Alfa Romeo,GT,3.0,WW au Maroc,Non,11.0,12000
6791,Marrakech,2019,Automatique,Diesel,134999.5,Volkswagen,Golf 7,5.0,Dédouanée,Non,8.0,1
6832,Tanger,2019,Manuelle,Diesel,174999.5,Dacia,Sandero,5.0,WW au Maroc,Non,6.0,10500


In [61]:
#remove all rows that has a weird price
data_scrap=data_scrap.drop(data_scrap[data_scrap['Prix'] < 20000].index)

In [62]:
data_scrap.shape

(4299, 12)

In [63]:
data_scrap=data_scrap.drop(data_scrap[data_scrap['Origine']=='Pas encore dédouanée'].index)

In [64]:
#convert the column Puissance 
data_scrap['Puissance fiscale']= data_scrap['Puissance fiscale'].astype(int)

In [65]:
data_scrap['Origine'].unique()

array(['WW au Maroc', 'Dédouanée', 'Importée neuve'], dtype=object)

In [66]:
data_scrap['Ville'].unique()

array(['Casablanca', 'Rabat', 'Marrakech', 'Fès', 'Agadir', 'Tétouan',
       'Agadir Melloul', 'Salé', 'Mohammedia', 'Meknès', 'Nador',
       'Tanger', 'Ifrane', 'Kénitra', 'Laâyoune', 'Ben Ahmed', 'Temara',
       'Dar Bouazza', 'Oujda', 'الدار البيضاء', 'Ouazzane', 'El Jadida',
       'Béni Mellal', 'Larache', 'Ben Guerir', 'Essaouira', 'Safi',
       'Ouarzazate', 'Khouribga', 'Sidi Rahal', 'القنيطرة',
       'Fquih Ben Saleh', 'الرباط', 'سلا', 'تطوان', 'فاس', 'Skhirat',
       'Bouskoura', 'Berkane', 'Sidi Kacem', "Kalaat M'gouna", 'Guercif',
       'Guigou', 'Berrechid', 'Laattaouia', 'Tamesna', 'Al Hoceima',
       'Kasba Tadla', 'وجدة', 'Ain Harrouda', 'Tit Mellil', 'مراكش',
       'Settat', 'Azilal', 'Taza', 'Saidia', 'Asilah', 'Bouznika',
       'Inzegan', 'Deroua', 'Khemisset', 'Sidi Bennour',
       'Souk Sebt Ouled Nemma', 'Biougra', 'Guelmim',
       'El Kelâa des Sraghna', 'Midelt', 'Errachidia', 'Khénifra', 'طنجة',
       'Echemmaia', 'Taroudant', 'Zagora', 'Oualidia',

In [67]:
data_scrap['Ville'] = data_scrap['Ville'].str.replace('الدار البيضاء', 'Casablanca')

In [68]:
data_scrap['Ville'] = data_scrap['Ville'].str.replace('الرباط', 'Rabat')

In [69]:
data_scrap['Ville']=data_scrap['Ville'].str.replace('سلا','Salé')

In [70]:
data_scrap['Ville']=data_scrap['Ville'].str.replace('القنيطرة','Kénitra')

In [71]:
#number of nan values in each column
data_scrap.isna().sum()

Ville                1658
Année-Modèle            0
Boite de vitesses       0
Type de carburant       0
Kilométrage             0
Marque                  0
Modèle                  0
Nombre de portes        0
Origine                 0
Première main           0
Puissance fiscale       0
Prix                    0
dtype: int64

In [72]:
data_scrap['Année-Modèle'].unique()

array([2018, 2022, 2014, 2021, 2020, 2019, 2015, 2011, 2017, 2008, 2025,
       2023, 2024, 2016, 2012, 2013, 1989, 2009, 2007, 2006, 2010, 1994,
       2003, 2005, 1996, 1992, 2002, 1987, 2004, 1997, 1999, 1998,    0,
       1985, 1991, 1988, 2001, 1993, 1995, 2000, 1986, 1990, 1982, 1983])

In [73]:
data_scrap = data_scrap[data_scrap['Année-Modèle'] != 0]

In [74]:
data_scrap.to_csv('../../data/scrapped/voitures_clean2.csv',index=False)

In [75]:
data_scrap.shape

(4257, 12)